In [ ]:
# install library
%%capture
!pip install sentence_transformers
!pip install datasets

In [ ]:
# load csv datasets
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/P_Text_Similarity.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
len(dataset['train'])

3000

In [ ]:
dataset['train']

Dataset({
    features: ['text1', 'text2'],
    num_rows: 3000
})

In [ ]:
train_sentences = []

for i in range(len(dataset['train'])):
  train_sentences.append(dataset['train'][i]['text1'])
  train_sentences.append(dataset['train'][i]['text2'])


In [ ]:
len(train_sentences)

6000

In [ ]:
# fine tuning tsdae model to adobt domain context

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Define a list with sentences (1k - 100k sentences)
# train_sentences = ["Your set of sentences",
#                    "Model will automatically add the noise",
#                    "And re-construct it",
#                    "You should provide at least 1k sentences"]

# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=6, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model.save('output/tsdae-model')

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.4.crossattention.self.value.weight', 'bert.encoder.layer.7.crossattention.self.query.bias', 'bert.encoder.layer.5.crossattention.self.query.weight', 'bert.encoder.layer.11.crossattention.output.dense.weight', 'bert.encoder.layer.7.crossattention.self.key.bias', 'bert.encoder.layer.4.crossattention.output.dense.weight', 'bert.encoder.layer.4.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.5.crossattention.self.key.bias', 'bert.encoder.layer.3.crossattention.self.key.bias', 'bert.encoder.layer.2.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.8.crossattention.self.key.weight', 'bert.encoder.layer.5.crossattention.self.query.bias', 'bert.encoder.layer.2.crossattention.self.value.weight', 'bert.encoder.layer.8.crossattention.o

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1000 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [ ]:
!huggingface-cli login --token <hf_token>

In [ ]:
# save to hugginface hub

from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/content/output/tsdae-model",
    repo_id="sam2ai/sbert-tsdae",
    repo_type="model",
)

In [ ]:
# inference

In [ ]:
# Define your sentence transformer model using CLS pooling
from sentence_transformers import SentenceTransformer, util, models

model_name = 'sam2ai/sbert-tsdae'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
def similarity_score(model, textA, textB):
    em_test = model.encode(
        [textA, textB],
        normalize_embeddings=True
    )
    return em_test[0] @ em_test[1].T

In [ ]:
# Two lists of sentences
sentences_1 = ["broadband challenges tv viewing the number of europeans with broadband has exploded over the past 12 months  with the web eating into tv viewing habits  research suggests.  just over 54 million people are hooked up to the net via broadband  up from 34 million a year ago  according to market analysts nielsen/netratings. the total number of people online in europe has broken the 100 million mark. the popularity of the net has meant that many are turning away from tv  say analysts jupiter research. it found that a quarter of web users said they spent less time watching tv in favour of the net  the report by nielsen/netratings found that the number of people with fast internet access had risen by 60% over the past year.  the biggest jump was in italy  where it rose by 120%. britain was close behind  with broadband users almost doubling in a year. the growth has been fuelled by lower prices and a wider choice of always-on  fast-net subscription plans.  twelve months ago high speed internet users made up just over one third of the audience in europe; now they are more than 50% and we expect this number to keep growing   said gabrielle prior  nielsen/netratings analyst.  as the number of high-speed surfers grows  websites will need to adapt  update and enhance their content to retain their visitors and encourage new ones.  the total number of europeans online rose by 12% to 100 million over the past year  the report showed  with the biggest rise in france  italy  britain and germany.  the ability to browse web pages at high speed  download files such as music or films and play online games is changing what people do in their spare time.  a study by analysts jupiter research suggested that broadband was challenging television viewing habits. in homes with broadband  40% said they were spending less time watching tv. the threat to tv was greatest in countries where broadband was on the up  in particular the uk  france and spain  said the report. it said tv companies faced a major long-term threat over the next five years  with broadband predicted to grow from 19% to 37% of households by 2009.  year-on-year we are continuing to see a seismic shift in where  when and how europe s population consume media for information and entertainment and this has big implications for tv  newspaper and radio   said jupiter research analyst olivier beauvillian.,gardener wins double in glasgow britain s jason gardener enjoyed a double 60m success in glasgow in his first competitive outing since he won 100m relay gold at the athens olympics.  gardener cruised home ahead of scot nick smith to win the invitational race at the norwich union international. he then recovered from a poor start in the second race to beat swede daniel persson and italy s luca verdecchia. his times of 6.61 and 6.62 seconds were well short of american maurice greene s 60m world record of 6.39secs from 1998.  it s a very hard record to break  but i believe i ve trained very well   said the world indoor champion  who hopes to get closer to the mark this season.  it was important to come out and make sure i got maximum points. my last race was the olympic final and there was a lot of expectation.  this was just what i needed to sharpen up and get some race fitness. i m very excited about the next couple of months.   double olympic champion  marked her first appearance on home soil since winning 1500m and 800m gold in athens with a victory. there was a third success for britain when  edged out russia s olga fedorova and sweden s jenny kallur to win the women s 60m race in 7.23secs. maduaka was unable to repeat the feat in the 200m  finishing down in fourth as  took the win for russia. and the 31-year-old also missed out on a podium place in the 4x200m relay as the british quartet came in fourth  with russia setting a new world indoor record. there was a setback for jade johnson as she suffered a recurrence of her back injury in the long jump. russia won the meeting with a final total of 63 points  with britain second on 48 and france one point behind in third.  led the way for russia by producing a major shock in the high jump as he beat olympic champion stefan holm into second place to end the swede s 22-event unbeaten record.  won the triple jump with a leap of 16.87m  with britain s tosin oke fourth in 15.80m.  won the men s pole vault competition with a clearance of 5.65m  with britain s nick buckfield 51cm adrift of his personal best in third. and  won the women s 800m  with britain s jenny meadows third. there was yet another russian victory in the women s 400m as  finished well clear of britain s catherine murphy. chris lambert had to settle for fourth after fading in the closing stages of the men s 200m race as sweden s  held off leslie djhone of france. france s  won the men s 400m  with brett rund fourth for britain.  took victory for sweden in the women s 60m hurdles ahead of russia s irina shevchenko and britain s sarah claxton  who set a new personal best. italy grabbed their first victory in the men s 1500m as  kicked over the last 200 metres to hold off britain s james thie and france s alexis abraham. a botched changeover in the 4x200m relay cost britain s men the chance to add further points as france claimed victory."]
sentences_2 = ["rap boss arrested over drug find rap mogul marion  suge  knight has been arrested for violating his parole after he was allegedly found with marijuana.  he was arrested in barstow  california  on saturday following an alleged traffic offence. he is expected to be transferred to a state prison while a decision is made on whether he should be released. mr knight  founder of death row records  served a 10-month jail term in 2004 for punching a man while on parole for an assault conviction. police said mr knight was stopped on saturday after performing an illegal u-turn and a search of his car allegedly found marijuana.  he is also accused of not having insurance. a 18-year-old woman in the car was arrested for providing false information and having a fake id card. she was later released. it was his second alleged violation  having previously served half of a nine-year sentence for breaking the terms of his parole. mr knight  39  was jailed in october 1996 following his involvement in a fight with a rival gang just hours before rapper tupac shakur was killed in a las vegas drive-by shooting. he was driving shakur s car at the time and was shot in the head. at the time he was on probation for assaulting two musicians. mr knight  a former bodyguard  set up death row records in the early 1990s with shakur and dr dre among his protegees. but the label has always been dogged by allegations it supports gang culture and fuels the east and west coast rap rivalry.,amnesty chief laments war failure the lack of public outrage about the war on terror is a powerful indictment of the failure of human rights groups  amnesty international s chief has said.  in a lecture at the london school of economics  irene khan said human rights had been flouted in the name of security since 11 september  2001. she said the human rights movement had to use simpler language both to prevent scepticism and spread a moral message. and it had to fight poverty  not just focus on political rights for elites.  ms khan highlighted detentions without trial  including those at the us camp at guantanamo bay in cuba  and the abuse of prisoners as evidence of increasing human rights problems.  what s a new challenge is the way in which this age-old debate on security and human rights has been translated into the language of war   she said.  by using the language of war  human rights are being sidelined because we know human rights do not apply in times of war.  ms khan said such breaches were infectious and were now seen in almost very major country in the world.  the human rights movement faces a crisis of faith in the value of human rights   she said. that was accompanied by a crisis of governance  where the united nations system did not seem able to hold countries to account.  the amnesty secretary-general said a growing gap between the perceived influence of human rights group and what they could actually achieve was fuelling scepticism.  public passivity on the war against terror is the single most powerful indictment on the failures of human rights groups   she said. ms khan said the movement had failed to mobilise public outrage about what was happening to the human rights system. there needed to be a drive to use simpler language  talking about the basic morality of the issues rather than the complexity of legal processes. such efforts could make the issues more relevant to people across the world  she said.  the human rights groups also had to recognise there were new groups which had to be tackled in new ways as power dripped away from state governments. al-qaeda  for example  was not going to be impressed by a traditional amnesty letter writing campaign. more also needed to be done to develop a human rights framework for international business corporations. amnesty international members voted in 2001 to extend the organisation s work from political and civil rights to cover social and economic rights too. ms khan said the human rights movement would make itself irrelevant if it turned away from the suffering caused by economic strife.  we would be an elitist bunch working for the elites  for those who cannot read the newspaper of their choice rather than those who cannot read   she said. despite her concerns  ms khan dubbed herself a  hope-monger   saying she was confident the passions of the human rights movement could overcome the new challenges."]



q_embeddings = model.encode(sentences_1[0], normalize_embeddings=True)
p_embeddings = model.encode(sentences_2[0], normalize_embeddings=True)
scores = q_embeddings @ p_embeddings.T

print(scores)

0.7630408


In [ ]:
# visualize embeddings

In [ ]:
# !pip install nomic

In [ ]:
!nomic login

                                  Authenticate with the Nomic API                                   
                                  https://atlas.nomic.ai/cli-login                                  
       Click the above link to retrieve your access token and then run `nomic login [token]`        


In [ ]:
!nomic login <token>

In [ ]:
len(train_sentences[:10])

10

In [ ]:
import torch
from tqdm import tqdm

batch_size = 10 # lower this if needed
embeddings = []
for i in tqdm(range(0, len(train_sentences), batch_size)):
  batch = [document for document in train_sentences[i:i+batch_size]]
  cls_embeddings = model.encode(batch, normalize_embeddings=True)
  embeddings.append(cls_embeddings)

In [ ]:
import numpy as np

embeddings = np.concatenate(embeddings)
print(embeddings.shape)

(6000, 768)


In [ ]:
from nomic import atlas

response = atlas.map_embeddings(
    embeddings=embeddings,
    name="Huggingface TSDAE Embeding",
    description="An example of building a text emb ")

print(response)

2023-12-19 11:24:49.796 | WARNING  | nomic.atlas:map_embeddings:95 - An ID field was not specified in your data so one was generated for you in insertion order.
2023-12-19 11:24:51.305 | INFO     | nomic.project:_create_project:797 - Creating project `Huggingface TSDAE Embeding` in organization `sambitsekhar`
2023-12-19 11:24:52.377 | INFO     | nomic.atlas:map_embeddings:111 - Uploading embeddings to Atlas.
2it [00:01,  1.28it/s]
2023-12-19 11:24:54.183 | INFO     | nomic.project:_add_data:1426 - Upload succeeded.
2023-12-19 11:24:54.192 | INFO     | nomic.atlas:map_embeddings:129 - Embedding upload succeeded.
2023-12-19 11:24:55.633 | INFO     | nomic.project:create_index:1135 - Created map `Huggingface TSDAE Embeding` in project `Huggingface TSDAE Embeding`: https://atlas.nomic.ai/map/9d544104-483c-462e-a70a-dfb65debb9e3/1c401aca-13bf-4432-bebd-627ef5d51b1b
2023-12-19 11:24:55.637 | INFO     | nomic.atlas:map_embeddings:142 - Huggingface TSDAE Embeding: https://atlas.nomic.ai/map/9d

Huggingface TSDAE Embeding: https://atlas.nomic.ai/map/9d544104-483c-462e-a70a-dfb65debb9e3/1c401aca-13bf-4432-bebd-627ef5d51b1b
